In [1]:
rm(list=ls())
dir = '/Users/sungjinpark/OneDrive - konkuk.ac.kr/datamining/appstore_2/3randomforest'
setwd(dir)
getwd()
load('cls_1_train.rda')
load('cls_1_test.rda')

[1] "/Users/sungjinpark/OneDrive - konkuk.ac.kr/datamining/appstore_2/3randomforest"

# 5 classes

## 'Education' 'Entertainment' 'Games' 'Photo & Video' 'Utilities' 'etc'

In [2]:
load('cls_3_train.rda')
load('cls_3_test.rda')
cls_1_train = cls_3_train
cls_1_test = cls_3_test
#str(cls_1_train)
colnames(cls_1_train)[4] <- "Y"
colnames(cls_1_test)[4]<- "Y"
##################################################################
# relevel categorical variables
##################################################################
cls_1_train <- droplevels(cls_1_train)    
cls_1_test <- droplevels(cls_1_test)
levels(cls_1_train$Y)
levels(cls_1_train$prime_genre)

[1] "1" "2" "3" "4" "5"

[1] "Education"     "Entertainment" "Games"         "Photo & Video"
[5] "Utilities"     "etc"

# Randomforest Tuning

In [4]:
################################################
## Randomforest
################################################
library(randomForest)
tune_mat = matrix(0,4,6)
colnames(tune_mat) = c("default","mtry:3","mtry:5","mtry:7","mtry9","mtry:11")
rownames(tune_mat) = c('mtry','ntree','node','test_acc');tune_mat

,default,mtry:3,mtry:5,mtry:7,mtry9,mtry:11
mtry,0,0,0,0,0,0
ntree,0,0,0,0,0,0
node,0,0,0,0,0,0
test_acc,0,0,0,0,0,0


### Default model

In [5]:
set.seed(1)
default.fit = randomForest(Y~.,data=cls_1_train)
set.seed(1)
tab = table(cls_1_test[,4],predict(default.fit,newdata=cls_1_test[,-4]))
ts.ccp = sum(diag(tab))/sum(tab);
#default.fit$ntree
#default.fit$mtry # 3
#default.fit$nodesize # 1
tune_mat[,1]=c(3,500,1,0.735007);tune_mat

,default,mtry:3,mtry:5,mtry:7,mtry9,mtry:11
mtry,3.000000,0,0,0,0,0
ntree,500.000000,0,0,0,0,0
node,1.000000,0,0,0,0,0
test_acc,0.735007,0,0,0,0,0


## Randomforest -Tuning

In [6]:
mtry=seq(3,11,2) # 
ntree = seq(30,300,by=30) # 10개
count_mtry=2
count_ntree = 1
for(i in mtry){
  #print("mtry")
  #print(i)
  ntree.mat = matrix(0,3,10);
  colnames(ntree.mat) = seq(30,300,by=30)
  rownames(ntree.mat) = c("ntree","node","ts_acc")
  for(j in ntree)
  {
    node.mat=matrix(0,3,3)
    colnames(node.mat) = c("node1","node2","node3")
    rownames(node.mat) = c("ntree","node","ts_acc")
    #print("ntree")
    #print(j)
    for(k in c(1:3))
    {
      set.seed(1);
      rf.fit=randomForest(Y~.,data=cls_1_train,ntree=j,mtry=i,nodesize=k,type='C') 
      set.seed(1)
      tab = table(cls_1_test[,4],predict(rf.fit,newdata=cls_1_test[,-4]))
      ts.ccp = sum(diag(tab))/sum(tab)
      node.mat[,k] = c(j,k,ts.ccp)
    }
    opt.node = which.max(node.mat[3,])
    ntree.mat[,count_ntree] = node.mat[,opt.node]
    count_ntree = count_ntree + 1
    print(ntree.mat)
  }
  count_ntree = 1
  opt.ntree = which.max(ntree.mat[3,])
  ntree.mat[,opt.ntree]
  tune_mat[,count_mtry] = c(i,ntree.mat[,opt.ntree])
  count_mtry = count_mtry + 1
}

tune_mat

               30 60 90 120 150 180 210 240 270 300
ntree  30.0000000  0  0   0   0   0   0   0   0   0
node    3.0000000  0  0   0   0   0   0   0   0   0
ts_acc  0.7426778  0  0   0   0   0   0   0   0   0
               30         60 90 120 150 180 210 240 270 300
ntree  30.0000000 60.0000000  0   0   0   0   0   0   0   0
node    3.0000000  3.0000000  0   0   0   0   0   0   0   0
ts_acc  0.7426778  0.7412831  0   0   0   0   0   0   0   0
               30         60        90 120 150 180 210 240 270 300
ntree  30.0000000 60.0000000 90.000000   0   0   0   0   0   0   0
node    3.0000000  3.0000000  2.000000   0   0   0   0   0   0   0
ts_acc  0.7426778  0.7412831  0.737099   0   0   0   0   0   0   0
               30         60        90         120 150 180 210 240 270 300
ntree  30.0000000 60.0000000 90.000000 120.0000000   0   0   0   0   0   0
node    3.0000000  3.0000000  2.000000   1.0000000   0   0   0   0   0   0
ts_acc  0.7426778  0.7412831  0.737099   0.7447699   0   0 

,default,mtry:3,mtry:5,mtry:7,mtry9,mtry:11
mtry,3.000000,3.0000000,5.0000000,7.0000000,9.0000000,11.0000000
ntree,500.000000,300.0000000,90.0000000,120.0000000,300.0000000,210.0000000
node,1.000000,1.0000000,1.0000000,1.0000000,3.0000000,1.0000000
test_acc,0.735007,0.7482566,0.7538354,0.7580195,0.7524407,0.7496513


In [7]:
tune_mat[,which.max(tune_mat[4,])]

mtry       ntree        node    test_acc 
  7.0000000 120.0000000   1.0000000   0.7580195

## Result 

- Default : mtry: 3 , ntree: 5, nodesize: 1

- best_tune : mtry: 7, ntree: 120, nodesize: 1

In [8]:
result.mat = matrix(0,1,2)
colnames(result.mat)=c("default","tune")
rownames(result.mat)=c("test_acc")
set.seed(1)
default_fit=randomForest(Y~.,data=cls_1_train,type='C')
set.seed(1)
tab = table(cls_1_test[,4],predict(default_fit,newdata=cls_1_test[,-4]))
ts.ccp = sum(diag(tab))/sum(tab)
result.mat[,1] = c(ts.ccp);result.mat

,default,tune
test_acc,0.7433752,0


In [10]:
set.seed(1)
tune_model1.fit=randomForest(Y~.,data=cls_1_train,ntree=120,mtry=7,nodesize=1,type='C') #default = 1
set.seed(1)
tab = table(cls_1_test[,4],predict(tune_model1.fit,newdata=cls_1_test[,-4]))
ts.ccp = sum(diag(tab))/sum(tab)
result.mat[,2] = c(ts.ccp);result.mat

,default,tune
test_acc,0.7433752,0.7580195


## Result

- Randomfores Importance

In [11]:
important_vars=cbind(default_fit$importance,tune_model1.fit$importance)
colnames(important_vars)=c('Default',"Tune_1")
important_vars=as.data.frame(important_vars)
order=important_vars[order(important_vars$Tune_1),]
order

,Default,Tune_1
,<dbl>,<dbl>
ver,65.44724,51.05558
ipadSc_urls.num,105.56901,93.03890
cont_rating,109.44690,98.71968
sup_devices.num,131.52843,124.54008
price,143.50830,130.80223
prime_genre,152.07793,150.35731
lang.num,180.48850,169.06017
rating_count_ver,370.77711,243.89448
sum_rate_ver,451.30312,334.84487
